In [1]:
require 'onnxruntime'
require 'rmagick'
require 'torch-rb'
require 'numo/narray'

def preprocess_image(image)
  # Resize to 224x224 using RMagick
  image = image.resize_to_fill(224, 224)
  
  # Convert to RGB array and normalize
  rgb_data = image.export_pixels(0, 0, image.columns, image.rows, 'RGB')
  rgb_array = Numo::DFloat.cast(rgb_data).reshape(3, 224, 224) / 255.0
  
  # Apply CLIP normalization
  means = Numo::DFloat[0.48145466, 0.4578275, 0.40821073]
  stds = Numo::DFloat[0.26862954, 0.26130258, 0.27577711]
  
  3.times do |c|
    rgb_array[c, true, true] -= means[c]
    rgb_array[c, true, true] /= stds[c]
  end
  
  # Add batch dimension
  rgb_array.reshape(1, 3, 224, 224)
end

def simple_tokenize(text, context_length=77)
  # Simple tokenizer that creates zero array
  Numo::Int64.zeros(1, context_length)
end

# def get_image_embeddings(model, image_paths, batch_size: 32)
#   embeddings_list = []
  
#   (0...image_paths.size).step(batch_size) do |start_idx|
#     batch_paths = image_paths[start_idx, batch_size]
    
#     # Process batch of images
#     batch_tensors = batch_paths.map do |path|
#       image = Magick::Image.read(path).first
#       preprocess_image(image)
#     end
    
#     # Combine batch
#     batch_tensor = Numo::DFloat.zeros(batch_paths.size, 3, 224, 224)
#     batch_tensors.each_with_index do |tensor, i|
#       batch_tensor[i, true, true, true] = tensor[0, true, true, true]
#     end
    
#     # Create dummy text tokens
#     dummy_texts = Numo::Int64.zeros(batch_paths.size, 77)
    
#     # Run inference
#     outputs = model.predict({
#       "image_input" => batch_tensor,
#       "text_input" => dummy_texts
#     })
    
#     # Get embeddings - ensure it's a Numo::NArray
#     batch_embeddings = Numo::NArray.cast(outputs["image_features"])
    
#     # Add debugging
#     puts "Batch embeddings type: #{batch_embeddings.class}"
#     puts "Batch embeddings shape: #{batch_embeddings.shape}"
    
#     # Convert to torch tensor
#     begin
#       batch_embeddings_torch = Torch.tensor(batch_embeddings.to_a)
#       norms = batch_embeddings_torch.norm(2, 1, keepdim: true)
#       normalized_embeddings = batch_embeddings_torch / norms
      
#       embeddings_list << normalized_embeddings
#     rescue => e
#       puts "Error in conversion: #{e.message}"
#       puts "Shape before conversion: #{batch_embeddings.shape}"
#       next
#     end
#   end
  
#   # Combine all batches
#   Torch.cat(embeddings_list, dim: 0)
# end




# Usage
# begin
#   # Load ONNX model
#   model = OnnxRuntime::Model.new('clip_image_text_encoder.onnx')
  
#   # Process directory of images
#   image_dir = 'house_data_png'
#   image_paths = Dir[File.join(image_dir, '*')]
#   puts "Found #{image_paths.length} images"
  
#   # Get embeddings
#   embeddings = get_image_embeddings(model, image_paths)
#   puts "Final embeddings shape: #{embeddings.shape}"
  
# rescue => e
#   puts "Error: #{e.message}"
#   puts e.backtrace
# end


def get_image_embeddings(model, image_paths, batch_size: 32, limit: nil)
 # Apply limit if specified
 image_paths = image_paths[0...limit] if limit
 
 puts "Processing #{image_paths.length} images..."
 
 embeddings_list = []
 
 (0...image_paths.size).step(batch_size) do |start_idx|
   batch_paths = image_paths[start_idx, batch_size]
   
   puts "Processing batch #{start_idx/batch_size + 1} of #{(image_paths.size.to_f/batch_size).ceil}"
   
   # Process batch of images
   batch_tensors = batch_paths.map do |path|
     image = Magick::Image.read(path).first
     preprocess_image(image)
   end
   
   # Combine batch
   batch_tensor = Numo::DFloat.zeros(batch_paths.size, 3, 224, 224)
   batch_tensors.each_with_index do |tensor, i|
     batch_tensor[i, true, true, true] = tensor[0, true, true, true]
   end
   
   # Create dummy text tokens
   dummy_texts = Numo::Int64.zeros(batch_paths.size, 77)
   
   # Run inference
   outputs = model.predict({
     "image_input" => batch_tensor,
     "text_input" => dummy_texts
   })
   
   # Get embeddings - ensure it's a Numo::NArray
   batch_embeddings = Numo::NArray.cast(outputs["image_features"])
   
   # Add debugging
   puts "Batch embeddings type: #{batch_embeddings.class}"
   puts "Batch embeddings shape: #{batch_embeddings.shape}"
   
   # Convert to torch tensor
   begin
     batch_embeddings_torch = Torch.tensor(batch_embeddings.to_a)
     norms = batch_embeddings_torch.norm(2, 1, keepdim: true)
     normalized_embeddings = batch_embeddings_torch / norms
     
     embeddings_list << normalized_embeddings
   rescue => e
     puts "Error in conversion: #{e.message}"
     puts "Shape before conversion: #{batch_embeddings.shape}"
     next
   end
 end
 
 # Combine all batches
 Torch.cat(embeddings_list, dim: 0)
end

# Usage
begin
 # Load ONNX model
 model = OnnxRuntime::Model.new('clip_image_text_encoder.onnx')
 
 # Process directory of images
 image_dir = 'house_data_png'
 image_paths = Dir[File.join(image_dir, '*')]
 puts "Found #{image_paths.length} images"
 
 # Get embeddings for first 100 images
 embeddings = get_image_embeddings(model, image_paths, limit: 100)
 puts "Final embeddings shape: #{embeddings.shape}"
 
rescue => e
 puts "Error: #{e.message}"
 puts e.backtrace
end

Found 5249 images
Processing 100 images...
Processing batch 1 of 4
Batch embeddings type: Numo::DFloat
Batch embeddings shape: [32, 512]
Processing batch 2 of 4
Batch embeddings type: Numo::DFloat
Batch embeddings shape: [32, 512]
Processing batch 3 of 4
Batch embeddings type: Numo::DFloat
Batch embeddings shape: [32, 512]
Processing batch 4 of 4
Batch embeddings type: Numo::DFloat
Batch embeddings shape: [4, 512]
Final embeddings shape: [100, 512]


In [2]:
embeddings

tensor([[ 0.0086, -0.0172,  0.0142,  ...,  0.0412, -0.0064,  0.0027],
        [ 0.0140,  0.0025,  0.0138,  ...,  0.0485,  0.0043,  0.0073],
        [-0.0073, -0.0085,  0.0317,  ...,  0.0320, -0.0004,  0.0363],
        ...,
        [ 0.0066, -0.0006,  0.0121,  ...,  0.0584, -0.0078, -0.0052],
        [ 0.0128, -0.0024,  0.0054,  ...,  0.0549, -0.0059, -0.0072],
        [-0.0009, -0.0002,  0.0173,  ...,  0.0421,  0.0040,  0.0087]])

In [3]:
# Function to find nearest neighbors
def find_nearest_neighbors(embeddings, query_index, k=5)
  # Get the query embedding
  query_embedding = embeddings[query_index]
  
  # Calculate similarities with all embeddings
  query_embedding = query_embedding.reshape(1, -1)  # Make it 2D: [1, embed_dim]
  
  # Calculate cosine similarity
  similarities = Torch.matmul(embeddings, query_embedding.t())
  
  # Get top k (excluding the first which is the query itself)
  values, indices = similarities.flatten.topk(k + 1)
  
  # Convert to Ruby arrays and remove the self-similarity
  similarity_scores = values.to_a[1..-1]  # Skip first one (self)
  neighbor_indices = indices.to_a[1..-1]  # Skip first one (self)
  
  return neighbor_indices, similarity_scores
end

# Save embeddings to file
def save_embeddings(embeddings, filename)
  File.open(filename, 'wb') do |file|
    Marshal.dump(embeddings.to_a, file)
  end
end

# Load embeddings from file
def load_embeddings(filename)
  embeddings_array = Marshal.load(File.read(filename))
  Torch.tensor(embeddings_array)
end

# Encode text to the same vector space as images
def embed_text(model, text)
  # Create simple tokenizer (77 is CLIP's context length)
  dummy_tokens = Numo::Int64.zeros(1, 77)
  
  # Get text embeddings
  outputs = model.predict({
    "image_input" => Numo::DFloat.zeros(1, 3, 224, 224),  # dummy image
    "text_input" => dummy_tokens
  })
  
  # Get and normalize text embeddings
  text_embedding = Torch.tensor(outputs["text_features"].to_a)
  text_embedding /= text_embedding.norm(2, -1, keepdim: true)
  text_embedding
end

# Search by text query
def search_by_text(model, embeddings, image_paths, query, k=5)
  # Get text embedding
  text_embedding = embed_text(model, query)
  
  # Calculate similarities
  similarities = Torch.matmul(embeddings, text_embedding.t())
  
  # Get top k results
  values, indices = similarities.flatten.topk(k)
  
  # Convert to Ruby arrays
  similarity_scores = values.to_a
  result_indices = indices.to_a
  
  # Get matching image paths
  result_paths = result_indices.map { |idx| image_paths[idx] }
  
  puts "\nTop #{k} matches for query: '#{query}'"
  result_paths.zip(similarity_scores).each_with_index do |(path, score), i|
    puts "#{i+1}. #{File.basename(path)} (similarity: #{score.round(3)})"
  end
  
  return result_paths, similarity_scores
end

# Search by image index
def search_by_image(embeddings, image_paths, index, k=5)
  neighbor_indices, similarity_scores = find_nearest_neighbors(embeddings, index, k)
  
  result_paths = neighbor_indices.map { |idx| image_paths[idx] }
  
  puts "\nTop #{k} similar images to #{File.basename(image_paths[index])}:"
  result_paths.zip(similarity_scores).each_with_index do |(path, score), i|
    puts "#{i+1}. #{File.basename(path)} (similarity: #{score.round(3)})"
  end
  
  return result_paths, similarity_scores
end

# Usage example:
begin
  # Load model and embeddings
  model = OnnxRuntime::Model.new('clip_image_text_encoder.onnx')
  
  # Either generate embeddings:
  image_paths = Dir[File.join('house_data_png', '*')]
  embeddings = get_image_embeddings(model, image_paths, limit: 100)
  
  # Or load pre-saved embeddings:
  # embeddings = load_embeddings('house_data_png.marshal')
  
  # Example searches
  search_by_text(model, embeddings, image_paths, "large kitchen island colonial")
  search_by_text(model, embeddings, image_paths, "white marble shower stall")
  
  # Search by image example
  search_by_image(embeddings, image_paths, 5)  # Changed from 505 to 5 since we limited to 100
  
rescue => e
  puts "Error: #{e.message}"
  puts e.backtrace
end

Processing 100 images...
Processing batch 1 of 4
Batch embeddings type: Numo::DFloat
Batch embeddings shape: [32, 512]
Processing batch 2 of 4
Batch embeddings type: Numo::DFloat
Batch embeddings shape: [32, 512]
Processing batch 3 of 4
Batch embeddings type: Numo::DFloat
Batch embeddings shape: [32, 512]
Processing batch 4 of 4
Batch embeddings type: Numo::DFloat
Batch embeddings shape: [4, 512]

Top 5 matches for query: 'large kitchen island colonial'
1. bath_114.png (similarity: 0.241)
2. bath_1150.png (similarity: 0.24)
3. bath_1013.png (similarity: 0.239)
4. bath_1169.png (similarity: 0.238)
5. bath_1059.png (similarity: 0.237)

Top 5 matches for query: 'white marble shower stall'
1. bath_114.png (similarity: 0.241)
2. bath_1150.png (similarity: 0.24)
3. bath_1013.png (similarity: 0.239)
4. bath_1169.png (similarity: 0.238)
5. bath_1059.png (similarity: 0.237)

Top 5 similar images to bath_1006.png:
1. bath_1111.png (similarity: 0.987)
2. bath_1149.png (similarity: 0.985)
3. bath_

[["house_data_png/bath_1111.png", "house_data_png/bath_1149.png", "house_data_png/bath_1141.png", "house_data_png/bath_1136.png", "house_data_png/bath_1163.png"], [0.9867925643920898, 0.9853135347366333, 0.9835642576217651, 0.9810572862625122, 0.980038046836853]]

In [5]:
require 'rmagick'
require 'base64'

# Function to display image in Jupyter notebook
# def display_image(image_path)
#   # Read image using RMagick
#   img = Magick::Image.read(image_path).first
  
#   # Resize if too large (optional)
#   if img.columns > 500 || img.rows > 500
#     img = img.resize_to_fit(500, 500)
#   end
  
#   # Convert to blob and base64
#   blob = img.to_blob { self.format = 'JPEG' }
#   base64_img = Base64.encode64(blob)
  
#   # Display using IRuby
#   IRuby.display IRuby.html "<img src='data:image/jpeg;base64,#{base64_img}'>"
# end

def display_image(image_path)
  # Read image using RMagick
  img = Magick::Image.read(image_path).first
  
  # Resize if too large (optional)
  if img.columns > 500 || img.rows > 500
    img = img.resize_to_fit(500, 500)
  end
  
  # Convert to blob and base64
  blob = img.to_blob { |b| b.format = 'JPEG' }
  base64_img = Base64.encode64(blob)
  
  # Display using IRuby
  IRuby.display IRuby.html "<img src='data:image/jpeg;base64,#{base64_img}'>"
end

# Modified search functions to display images
def search_by_text(model, embeddings, image_paths, query, k=5)
  text_embedding = embed_text(model, query)
  similarities = Torch.matmul(embeddings, text_embedding.t())
  values, indices = similarities.flatten.topk(k)
  
  similarity_scores = values.to_a
  result_indices = indices.to_a
  result_paths = result_indices.map { |idx| image_paths[idx] }
  
  puts "\nTop #{k} matches for query: '#{query}'"
  result_paths.zip(similarity_scores).each_with_index do |(path, score), i|
    puts "#{i+1}. #{File.basename(path)} (similarity: #{score.round(3)})"
    display_image(path)
  end
  
  return result_paths, similarity_scores
end

def search_by_image(embeddings, image_paths, index, k=5)
  puts "\nQuery image:"
  display_image(image_paths[index])
  
  neighbor_indices, similarity_scores = find_nearest_neighbors(embeddings, index, k)
  result_paths = neighbor_indices.map { |idx| image_paths[idx] }
  
  puts "\nTop #{k} similar images to #{File.basename(image_paths[index])}:"
  result_paths.zip(similarity_scores).each_with_index do |(path, score), i|
    puts "#{i+1}. #{File.basename(path)} (similarity: #{score.round(3)})"
    display_image(path)
  end
  
  return result_paths, similarity_scores
end

# Usage example:
begin
  # Load model and embeddings
  model = OnnxRuntime::Model.new('clip_image_text_encoder.onnx')
  
  # Generate or load embeddings
  image_paths = Dir[File.join('house_data_png', '*')]
  embeddings = get_image_embeddings(model, image_paths, limit: 100)
  
  # Example searches with image display
  puts "Searching for: 'large kitchen island colonial'"
  search_by_text(model, embeddings, image_paths, "large kitchen island colonial")
  
  puts "\nSearching for: 'white marble shower stall'"
  search_by_text(model, embeddings, image_paths, "white marble shower stall")
  
  puts "\nSearching by image index 5:"
  search_by_image(embeddings, image_paths, 5)
  
rescue => e
  puts "Error: #{e.message}"
  puts e.backtrace
end

Processing 100 images...
Processing batch 1 of 4
Batch embeddings type: Numo::DFloat
Batch embeddings shape: [32, 512]
Processing batch 2 of 4
Batch embeddings type: Numo::DFloat
Batch embeddings shape: [32, 512]
Processing batch 3 of 4
Batch embeddings type: Numo::DFloat
Batch embeddings shape: [32, 512]
Processing batch 4 of 4
Batch embeddings type: Numo::DFloat
Batch embeddings shape: [4, 512]
Searching for: 'large kitchen island colonial'

Top 5 matches for query: 'large kitchen island colonial'
1. bath_114.png (similarity: 0.241)


"<img src='data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQE\nBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/\n2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQ\nEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCAEAAQADAREAAhEBAxEB/8QA\nHgAAAQQDAQEBAAAAAAAAAAAAAAMEBQYBAgcICQr/xABMEAEAAQMDAgMGBQEE\nBAsGBwABAgMEEQAFIRIxBkFRBxMiYXGBCBSRobEyFSNCwSRSYtEJMzRDU1Rz\nkqKz8BYmJ5Ph8WN0gpSywtL/xAAcAQEAAwEBAQEBAAAAAAAAAAAAAQIDBAUG\nBwj/xAAqEQEBAAICAgIBBAEEAwAAAAAAAQIRAxIEIRMxBQYyQVEjIjNCYYGR\nof/aAAwDAQACEQMRAD8AmvyELiZC9qrSizIkuAk46ZOMmTkHDjqU5AdVDtsq\ndvSpW1GlTjKNOMZHSLNDHUSMvxGHGc5lgyYUH+23lyWsKVxWlCvSzTjPGStC\nSpjB/UuByIuZZVwCwlf3NxsdjG/oLKrfAUqkpHxOUFewHPHbz88pEh94c8Pl\nrT/tXcpyq3VVyLJHqDGOO2OTjCPBjCyjFKwE5zQziMTEYnBEPIDgNXoc0opl\nV551VFh3S6gI9SZ1KDXeLvdLWEJWMCUZOJScrF8uDufPPGpyhDfY/Dm40/El\nr4g8QUJwoUsznGriLkJdD05yYkjz6apIsl7qs7xutvt9a+pU7S4nmTC4CVSQ\nqR+WXHbv2741emknXpW1elS26AUGNVISzicyIxYZkKChh5yGcK81TppCcril\nC22+5pW05znazqQJTn1xBAEDGOc8HocGWzR9GMrbbq9hQjXqXPvZR6qyYmoL\nLK8gIJnyxw9iC8yyjCFa6alOpUjFPiZMhMC4XK+mE+boHu3kYVGnRq9WfLPE\nsHOMPcz/AB66LpCVJFMy7+WihUor3m/q6DcpzDib+ui0renGUOWavlobKRrV\noZSrIfq6JNbq8jQp5mzlNemEIuZTXsB5v7euDnRGze2o14ddzW+OvPEiMVxE\n8gXuGe+OVXAcaItJrXqLKVatHPHMCOPpkz/J9NTpS0nV6oRf7x+E4c9/m/c1\npA0rE4UZPUrGCp8057p551MDK2pViM63vUbgjNPRx5+vrqkmojWz2zs5Zj1S\neEc588/786zuXtrjHwI1mkaA0BoPtHd7NCTLrox5XNSkYyfOPB+mHzfnqoi7\nnbbuhRnKMmpSlgJDmI5zxn+lcfJcc+TouQ2u1lYSqVoU6lanVkEi4quIuOJR\niHKA5XA8Y5OZpYR3OkS3Taq3RiEbkOl5MogP2yc6rUaXO5KdOdGhGZKFOjCM\nU5yYz3O/PnqTTelAe3b5aGjiBjBnt66KnVNiORz56uiw13uN1X2u5p2USVf3\ncug9XHbUWoh94Sq7xV8DUrjc68bmbOcKDUMMYQ+FJvnlHy4Md+wixztJbRo0\nNyrws6NtcYq0oINeMlPjyn9AZxgzntwBpTaILe/3C+uq8j31SM4SI5lGC54m\nLzxgFOwLzgi1Ttatr2qdvMalSE1mz4pgRkgKJyuA8vLSQ2kqu1U7W195ThVn\nSjGcSnBcSTCqg9WXjKBg7o41aRBptdhG4u4bhK2hRhCJPpqTJzqKYYphQzxz\n5fbTQc7ZO53jf43dKhOFjYE4zqzix97NMdER5wZy+mOcZMtCwSIyVQ58tVAR\necc6uM9MvTVRsQU5EdQjZrcXcY1JW1nTa9xEcxP6YvzQcfQ55PJySttiFnGy\nzeXrO5uJCLCGUO6RjlwcduV4yrq0iGkrqpMf9FrQ+LHYRPJ7iKc9n076IsZ/\nLhjEpTHn4ue/1ONW0qb1oCMc4zIHny8/4dSkzveshCED/jZkFz2H7c6AhbEf\nd0STLoiHbuBg1XK6icUk0vdEIfT+dc33Wj8+OgNAaA0H3JjQ95XuITioVpdK\n98cAnydas6Tq7bBqLSUnIQQOeMomMJ2znTdTKjrvZac3M6JDp5yRehe/J3PX\nzPkd9ExCbrZ0qNtO1u0654qUZQFlTRznJ3OP5O+QLJSwqO47ZSqziQr20SEo\n8GYZ4Qe+F+qJ6Opk0H9nDrnGHISkRO7yuDt89UyosniTwN4h8KxoS3mxnQK8\nWVPqwiGMnHmZMncyeuq459kWIalT7JI788a1s0rcTuEI5HHfu6sqf2fuqFCt\nZVKMaltcLKpT6sfEmFE7KBk88fNyGzbWlCxjY2kahQikiGOqWByxVco+ZnGi\nZTqrYD7rcLmPTTuYNTrlOH+jzkC9EukQcIgnKcd8El7evXrS6LWnGeJEYzhB\n9zIydpLyYzkBFUHImgd3dGruVSE6FCoUKVXNSNXqhCEIqdRiQnUPGBe3ougx\nSua97dT2/Y6tIpUszr3TTWMFDERyjJw+eE7dnQS+2W13Z0Z07q9/Ms5siXuy\nAZ7gC/voqdkPXH20GlWra28SdxcU6UXzqSAf11cJTv4OS0oVa8hRkQYwOOFl\nLGT5mdRQFpd3cgvbgpQ4fc26rhTvPhxkxkx5jnVQtVtLWhawt6BK2hEDppYH\nAYwp5PHAmolAVKMaBQ6K9UEMzgiq8LkzjnuGflqRjMSJ1QmJkRRz83H/AK9d\nAhWekegAzkDyNXDGtP8AvAznAyeOcv8A6dA2YQrXkVlPpgMvkIncx350Duzo\n9dx1Yzg89Z8l2tiXuZZqGXnIazxi7892qA0BoDQe8vZb+M72n7JGNn7Qdlpe\nKbHr5uqUC2vaYoOGIUqgHYYxV7z1E5oXF6s9nPto9mftPaUPDXiKnDcmLKW1\n3wULuLiOQhJxUDjLBnEXGdaY8kqNOhzpRUjI5e3HOeqJ/m/V9dWSQudktagz\nlQhzHGQwnd+mO+gib/bqtriVKlUqDk4wA+jzx5ds6m0NLaV3a1o1buh7iAZX\nlRxw441SzYsO5eJd5322oW9/u1S8p0DppQq1V92YxwS4DAdvQ1GOHURZOUJH\nUJ80761+w4hUlkM8Oo2pYdUJyZHmfTvqdqpGiLIcp9NSHsbShWjCFemVIQyx\nhPmIvdw8ZdEyn1OzosI0YUowhH+mMTpD6BjGraNlJbJt1arCrWt2pOPJ1Tkj\n9RcOmjaSoW9OhTjRoUo0qcc9MYRAMuXAcGVX6upQXjHtxnBoFoUsj1GgxZWV\nn+br1ZW1LMA6pdBlekxlxy8nfQI0aNepXa35i4qYmsYxYkImXjGQcevK+uo0\nHNxbwhIk0ypOSylLrY8gAieZg8ztx2NNDHu6MWP91TcmJLEVfq8uo0E6s6FO\npnJ14yxZZyY4AXj7Y++p0GM63VipMwnAZf8Ad++pDKtdYz1z83GO7oGE7zNS\nfPbBnJ2DP+boFrLPuZ15BmqsovngAP1xnQSFjKMYTqODLw6plFsWlRZTJd/i\nOfvrPel358tZg0BoDQe1fAXiXw1tlu09x2ilWXJlBx9s68nl482mNVj2h3Hh\n+7uZ3u025bThIlBhkYo5ETsjhz6mdb+PM8ftGWSyezH8ZPtU9nlahtniG8PF\nezQnGLS3Ka3NOI96dwZmJkfjKnbARzk9CZqPXnsx/Fd7G/arG3srffjYd3qy\njCO37yxoynNcEKdUWlUkrxEl1uOYmtN7HYq9CLhYhlO53H5effQMbrbKc6bT\nAlCWf7uQ47eWOTt3PTQQ13tdaLIosjIHu5ZeB8scL9Aee2gj7Sd0Xbbz+CmY\nc9fxJnHEXOX5cvGn0HUblhUhKrKNO0errrTMTiigYHHOMnPZO2dNhzSu4e59

2. bath_1150.png (similarity: 0.24)


"<img src='data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQE\nBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/\n2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQ\nEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCAEAAQADAREAAhEBAxEB/8QA\nHQAAAQQDAQEAAAAAAAAAAAAAAAQFBgcCAwgJAf/EAEUQAAEDAwMCBAQEBAQD\nBgYDAAECAxEEBSEABjESQQcTUWEIFCJxgZGhsRUywfAjQtHhFiRSFyUzYnLx\nNDVTgoOjY6Kz/8QAGwEAAgMBAQEAAAAAAAAAAAAAAAMBAgQFBgf/xAAwEQAC\nAgICAgEDAwIGAwEAAAAAAQIDESEEEhMxIgUyQRQjUWGxM0KBodHxFeHwcf/a\nAAwDAQACEQMRAD8A4VSEOPQIJUSM+sGB+mvHYzE9X/mJJd9uV172ineVOsL+\nU6aGpbmFBCEJShYGSYJIJHbJAgktqj8TPZL5EOAU2sqUokEAD2if9dNyVRLt\no3kIi2PuEAklog4Hcp/qPufbWaxZGxJnTuycKJB7yR+40nKiNwfbranrhSB6\nmbdLzCgtPSJKkz9aQDySmYH/AFdPpqyakHViq3W5KWg4paHQsAhaVyFDsZGD\nI7g6SsoYkODdMwwlKuogSDhckEZHPuNRKZLWRE2005W1bDbhgqS8lJUJTIgp\n98oJOOVaq3mIJG82dlRDriXEZwQpZzETIAHB7/6aM/EGjQq2NUJFQioXDTiH\n4mRCFBR59gf10yDeCskS4NqSoJ6ZIJEJzP5ao5PJMUPVput327Vt3ew3Sstt\na0khNRSPKadSOSAtJBAxxOjtJEyjGQ8XTxe8T71ROW26+Im6KukeT0uMP3Z9\nbax6FJWQR9xqHbJlI0wTykV5dPqudGhKAGQw+pZ7pWC2Ee2QXI+x99XjLRc3\nU6GUpgVBSVZISSJ7yY/vOhSA+1ZoWmHXlhbriGypKBkqIGAJHJOBnkjU5ZXA\n2vXH5ajFIjy1hlASVxBcVGVexJkn9NGWQR41DqnS70JMkmIGjLKJbH60U7bi\n0OOsIUGh15EnrIwRicJJHJB6iIkaMsuh+RUhSDCVxJyqZ5/P89VwyWjBbzh+\nlC1CcHJH9/76MMEYNOJBIbbWpZMlSiTn2k9z6d9VLPAuVUM29hS3XQFqzJVx\n7D99WUckd8CahebuzLtQ251pQ6WiRPIAVH66pYsME8mFajoUpIkiTj0g/wBn\n8dMXorI0tLIQemQRE/jqxUeLQhSnglZJBUBMwOCcH8tLbwXgee+veHkQ0AGg\nA0AXcUpFUjp+lSJVzyIKT+Mka8hH7T1S3Ifdk3Z2luKaZ2pcbp1hbf8AOekE\nwDKeCCJB/wBtOhLCEzjmRp3ptY2Nxu5USFrt1SrpSRkNLiSgkdjkgkAxjMSY\nZRoiiqhVKStuZSQpJByCIgj041aMOxZPGxbS+K5pnUUtbTeW7kdYwJBI+4Jj\niO/Ory+muS7IouWs4JRReMVMGUtrqGVJGIccB9u86Q+BKI9cpMyZ8VKFt1xC\nXabyyorA6xAnJA/GcdgY0p8KwFyDenxRt6wFFdOCMyFjSpcKwv5xPW+ItMbl\nT3CmqGkh1pbD6QoH0UFxHqI9p02HFscNlHeb2/ExhgdH8TbCDJ6Q4kD9tVjx\nbGsArzCt8S6Oton6VNXTS60tsEuCSSCP66vHiTXsiV5ImfGG3vNpUaqnQVAE\nhTokHnIxqr4c8lo3is+L1tWgf87TwBGHh+2qy40gXIiJleLFtEj5yn9v8Uf1\nGoXEkXXJiNj3ihRKr3qk3GnDSmWkJSXUgBQU4Sc9yFD8tOjxbMeinnRsHihb\nEn6a6lJJmfOSJ0Li2ZDzo0V/ihbFFlk1lMqV+Yr/ABhgJyCCOCF9BjuAdH6O\nZXzobazxHoVDpTWMEDH/AIo0fo5lfOIWfEKhLqUrraeJz/iiP17av+imR5iS\n0XixbKRtLIuVEQkY+tIMnJPPcyYHro/RTJ85sc8YLecJr6PPfzkD+uo/RTLO\n80/9rduB/wDmdIP/AMqT+2j9FMFeOa9132ro019qcbCFEpCinqzEyDMfvpHj\nwyzmR2vuW5qyTUVXWRyFAiPzOrKKFuRPvCF+oe2tW/NlIWi6OIwIEBlox+p0\nm+OGN47yiT18Ekf9UngY5/11C9FpPYlYQCTEwP8AfUkZJFZGwqOpKZWRHsAD\nn8f30ibwNgjzs1788gGgA0AGgC7GUpXcFpUojpQYTkCJiRP2/bXjlr5Hp032\nNNouDRuL1vKghxK8dRgEQMj+unWQk490EWu5Y1hqKSoadtF1S0ulqW+hxLq4\nAEzIJIjMEEGZziJGdWtsY60yv90WNVkuLtG2sutBRLToBAUkH1iCRwSMHB4I\nGtUJiJxcStLwHnryFPLUsqcQmVEk88ZzidduiWazmWRxYOqKVGE+XkHMp1ml\nI0w9ipLCUqCQhPSckxkH00luRfLNzTbYgGAZ9NL7SDZvDLRAAIMf+XUdpMnq\nffl2zM/tozJE4D5NnH0ZBkcCfw0dpIPGbmaIASQABiSJnUOyRKgKm6RBRmSJ\n7AZ0pyZPU+OUMLjygEdMyTmfQjgalTZVwEr1AjpILQJInCgJ01TkiGjUqhQQ\nFFuFD1yNW7yZHU0mianq6M+wiNHkaDBqfolghCGQvqzJOAPc/lpkZKRGD6i2\ntEAKb+rkg8D7HUOzAezNVuYHKEj2GT+2q+Usoms26mmfLMR21dWsjLNAoGvO\n6W2uO/TB47GdS7JEbOjrBamKTbDDXliA4Up7wBIA/TXHm/3GaorRhVUbHlFR\nZEAEEewGrRZGR38LWB/w1cVIgReXRP8A+Fkaz8l/MbQtEkrac9UmDH7TzoLP\n2JWWClzpIBAIP49/6aiRBJbKgtuoQUiRgwROcmD6wOe86zSZoho84tfQzxga\nADQAaALrZcK6xRC8yRM4A5JP468fJdY4PTqSk9EbuFJUIviy2VeYXEFMAzJA\njA9ddCucXTgQ0+5LDcLha6dk1yPJWUz9JBlMkAyCRzIjkEH01zo11t/E1NuH\nsWu7vst7t5t90qUoIHUy6AOpKpxBwIn8xjTvHYvSKeRSKtu1Kt2/IZYhZUpK\nklBkEDkj1wJA5zxrs0aq2c23/EHOmaZcAUiqQoCOFA/trJPKHRWxchpgj/xE\nzwMT+mlSchuUKUUqCJAWQOSGlGPxjVMyJ0fXPlmwoB8hSRJBbI/DIGpSkGRf\nSWx1woUll/60hQ+jkH7TOol2K5Fw2xcHG/Pat1wUgHp6xTKIByew/uNL7SGY\nFVPsy/vSWqGpECfrbKDHrBIx76W3IuoZFDe1rpTOJaqKWr61iQltkrJEniAZ\nODjnGly7kod1bQuTrP8Ay9trQpXdTCkk+8EAxnS8zJ6oZXdp3tIWtVK+EIJC\nlKTABGeSY1pUpsp1I1dvmre4aZNK71kSFqT0oInMEmCeMDWiCk9ipvAiZDaC\nRUViFOrzBMADngwTjV9SKNm9CadZANSgKJgQR/rqri4+iYik2hxC5Da1CMEI\nJn8hpLsGKBg5SeUelxKwfTy1T+2oTyEngTOBlBPmFaT/AOZpY/casuyKZNbX\nyqlkh0EJ5BB/bUylJFls6UpqPydu0TQwQAVCCDMSZB4MnWBbsZpawhBWMSyt\nRSCAkj0/fT4wM+djz4UUqV2qss7ySl9dxdq0oUkgqa8tpPVxxIjXO5UvmbaI\n6JJereWHCpHGRxM5/TURnks47EbDZEqUnJkyfXV3LJRIc6ZCgjM/UADBGFT+

3. bath_1013.png (similarity: 0.239)


...

4. bath_1136.png (similarity: 0.981)


"<img src='data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQE\nBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/\n2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQ\nEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCAEAAQADAREAAhEBAxEB/8QA\nHgABAAEEAwEBAAAAAAAAAAAAAAQCAwUGAQcKCAn/xABFEAABAwMCBAMGAgcE\nCQUBAAABAAIDBAURBiESMUFRBxNhFCJxgZGhFbEIIzJiweHwFlJT0RckJkJy\nc4KiwgkzQ7LxVP/EABoBAQADAQEBAAAAAAAAAAAAAAABAgMEBQb/xAApEQEB\nAAICAgICAQQDAQEAAAAAAQIRAxIhURMxBEEiFCMyYQUVQjNx/9oADAMBAAIR\nAxEAPwD9U0BAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQcIPKwgICAg9VCAg\nICAgICAgICAgICAgICAgICAgICAgICAgICDyroCAgIPVQgICAgICAgICAgIC\nAgICAgICAgICAgICAgICAg8q6AgICD1UICAgICAgICAgICAgICAgICAgICAg\nICAgICAgIPKugICAg9VCAgICAgICAgICAgICAgICAgICAgICAgICAgICDyro\nCAgIPVQgICAgICAgICAgICAgICAgICAgICAgICAgICAg8q6AgICD1T8Q6lU7\n439klMjup7T2WU2TvEapsneGqZHdR2xTqmR3U98TycQ7qdmqZHdV7w1XGyds\nQ2TtiOcjunfD2GQnyYT9jjbuE74+zye73Cd8fZ5Pd7hT3x9nlzkcshO2Ps8u\nPd7hR3x9nk93uE74+zye73Cd8fZ5Pd7hO+Ps8ucj+8E74+zVMj+8E74+zVM/\nvBO+Ps04yP7wTvPZpzkd1PeGqZHdO8NUyMbnKTKGjICmZSmq8rCkEBAQfpLF\np2jqAQ2hiIJO7Ys+vVfNTK+3ZMYuP0pamNHm2pgIHN0TQfU4wVpM77W6xFOj\nLBUucPYqZp/5DSfyVvlyNYotT4eade7AtFMevvU4Hy3T5cjWKFN4UaVlHFLa\n6AZ34fJbnHbYJeXI6xHZ4ZWamBZQUns5JzxQPLD9iMc1HzZK9Yi1fhxUxyea\ny83ONzeRNc94HyLjt8lF5at1iBNpLVbCWUGo5TnBwaOmec9yXRkk/PdV+Q6x\nSy0a9oME19uqT19qtrBt29wNU98VdX0sGXxDikBbQaXma0ZPBRSMcfn5hx9E\n74mr6XJ9Uaqoo8P0JbKl42cW3Ax5z2Hknl8SneX9o6xdptaVLYw646AnLx0p\nnwTD5cRZv8u6je/2vMY4Z4g2QSCOq0NeaVhGTJJQwuaB8I3uPTsp1fZrFdd4\nh6AhwHRzse/9prrRUgD4kxgde6avs1imx620A6Fj3Xyz0wcdmzzRxHr0cQfs\nosy/RrFk6C46Yu2H2yrtlYBtxQPjkx82kqm8jWKVJBbsHzKWD3dvejG31Cd8\nvZrFbNFbSMGipgRy/VNB/JO+Xs1iofSUL930lOT3MbT/AATvl7NYqXUNscMO\noabH/Jb/AJJ3y9msUeWz2YuLja6YE9fKAz9AnyX2dYivsFncSWUETDy5Z/PK\nfJfZ1iybHRDLo/dx0AH8kmeXs6xZfZiDxMdCR1EkDST8znKtMr7OsUvo6qNu\nGU1E8DfHlNH8Fa5U6xHdTytcHTWWmkAG5aGgqlyp1iLI2lJzJZCw55BoI+4S\nZXSesYy7C0Cgqz7E2KQQSFp8poIPCcYx1yteO1W4x8cL6BxCAgIP06joC8Db\nB7YK+X1euno0/D3sOW5B7jIzhTLqaRvSXBBXOw1lROADtmQgfYrTHyrlIltt\n9W/Dnuy4Dm4B2frladFZtbqYKxrQA5uAcHDGjb5BV17W2xsrZIwXOgY44PME\nE/QhY5an0tEGomc8kmmYANjjIP1JKz7ZCE+qjDcGCQEbDEuP4FVudEY1kTnB\nzo3ADqSHn74Vbyz0K5LlSlpHnVAHLZgA/NTObEWRUUgkDswAYGDKwuJ9dgQr\ndxHqJbfM4h7mOztwtZjPrkgD8lPyaHMcFsDCBBTsA/vSAZ+QO6TkFiSht8sg\n/VhwLs4YziB35Zxsr9pVdKKnT1rkJc63ngIwPMaME+uRnunaJ0xVRoPS9Y0t\nqbZTPOTt5AOc/EK0yhpAj8MdMUsjpaG3QUb8/txkxO+RaQford6jqol0TLHI\nHU+pL2COXDdKhwPphziPsnenVXJYNYQt4KTWVfEGnk6KnlPwJdGT907zH9Jk\n0pEHiLSxlkN4o6onbjqaM5+P6tzR9lFyxy/Ra4bc/EKmGa6ktVWByEYkgz8y\nXprFXdUDVer43gz6PiMQ2LoLgHux6B0bR9064m6qOupmO4anSd7YP7wZC5o+\nYkz9k6U3XEviNp+HhFU2405zyfbp3Y+bWEfdTMabq63XukHtBff6KEkZHnyi\nIn5PIKnpVrdJVLfrNcQXW+4007cYJika8H4YKpcYS7VSzsLi3jA7DuFFtSxl\n6kjNqrBxAA08ox3PAVphb8kRt8UL6NwCAgIPQfUeAPhVBOGs029kZO/DXVBx\n0zlzyvJ5fx5jXTx8nautr34KaXtl7qbfNVV7GMcXR8EjcGM7tILmkk4IB9QR\n0Xmcn8c9OqY7VaL8ErJcrvSU18nrYqSrqJaWOWmmYT5rTkA8UYAJaWDAznjB\nBG4Hf+Pxcd+6x5Mbi7ii/RG0O+EFl9uoJ5cXln8mhej/AE/HpjObXhgbp+hm\nyV0j7frhoaSSyOW3ZLR0BcJd/iAPgseX8Pv9Hyuh9beB+q9JXxlku9PBGahz\nm01SHPME/CCSWuDTjIGcEAjkQCuDP8W8X20nK1Gv8K9QxMc6JlNMQcgMmAz8\n3AYXJeHJbs1+fw31o0ECwuyADn2mDGOu/Hn7Kl4sjsw9ZoHWEJPHYKnPUMw/\n/wCpKzuGXpdjnaS1RsI9MXc5GctopSMd8huFHTL0MTcKKutjw250k1I48hOw\nxuI+DgCs96EQOimcDE9rnDcYIJUWgWEAkbj0UyUccJI3GRjqVfyOC1wAc1xH\nQEbJbRV7VWtaR7VMB6SEfkVEtFLaqcYJk4iOrwHH6kFW7ZJ24lrqlzSHOBHQ\ncDR+QCdsjbg3KRoLRTwgEDf3s59RnCvOW1DiK4kEmVjnDGwY/hGfmCovPcSx\nz+LQudg05YAeeQ7+AU/LikddKU5DnVAGP8NuP/tlW+XE8KRVUErC2SSKNuOs\nZJJ+AB/NWnJL+xbc21yADhifnkX+4CO++FeZT2I01rsk7yHtpX52w2Rrj9AS\npmc9q2bYiu0Fp6tzJPZoCCebqdpyfjhPkv6NMS/Qen6fLbbx0jtyfZZHQkH4\nscCEvLn4LjWPuOj5aWiqZ2X25hoie4smqnzBwDSSCHk4BGR0K2w5c/kV618p\nr6BxCAgIPStX8JY7fcZwc7Lj/I8tsJrJ13r23uqqWG8xAebSkQy46xknB+Tj\n/wBx6BeH+S78MtZNWi1FcrNNTmBhmgmcG8Bd+zMBljhscZOxxjOGnm0LHh5O\n30vyY+H0n4e3EXLTNBWOqjPLLFmR+TjjyQWjPQEFvy77n6T8TLvhp5fNjdtq\nc9rRlzgAe66fGLNjL7ZLJqG3ut97oIKumd73DIM4OCA4EbtIBOCCCOhVMrjk\nOldYeD9jp5HzWG5S02S4mGZokYCAMNaRhwAxzJcT1Xgfk8/T6dvFjco6uuun\nLnbnv82FsjR1heHA/DOOy4cPzMd/ya/FprdXUSQE4oKsuA24WsH8Va/mY/o+

5. bath_1163.png (similarity: 0.98)


"<img src='data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAMCAgICAgMCAgIDAwMDBAYEBAQE\nBAgGBgUGCQgKCgkICQkKDA8MCgsOCwkJDRENDg8QEBEQCgwSExIQEw8QEBD/\n2wBDAQMDAwQDBAgEBAgQCwkLEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQ\nEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBD/wAARCAEAAQADAREAAhEBAxEB/8QA\nHgAAAQQDAQEBAAAAAAAAAAAAAAQFBgcCAwgJAQr/xABCEAABAwMDAgMGAwYE\nBAcBAQABAgMRBAUhAAYxBxITQVEIImFxgZEUobEVIzJCwdEJUuHwFiQzYiVD\ncpKTovFlgv/EABsBAAEFAQEAAAAAAAAAAAAAAAABAgMEBQYH/8QALxEAAgIC\nAgICAgEEAwABBQAAAAECAwQREiEFMRMiMkFRBhQjMxVCYXEkQ1KBkf/aAAwD\nAQACEQMRAD8A5G6dbe7LRc1rQFJcfaAk+iVT+uuOyL9SSPR4U6Q+7p2DXXnZ\nhatbLoC6tC3S2CZSlK8EDkElJziQDzqfAt+zZneTq5QK1oLf1B6e3D9p7frK\n+jeaEl2lWUns85AOU4yDIPnI1szjVkx1IxKXZjvlEtLp97Q+2G6aptfVfZpu\nvjglFxpljxW1mT3LaUCFmSCCCkpIkZ1j3eKW91m7jeenFcbDtP2c6/pBbbTV\n1nSq90V1NalBrgHlJqG+0kAKZVCm0yVQSIOfeVAjEv8AmpeprRedteavqy5L\njats7yolW+70dPUpXktvpBM+oJ4IPBBBESNQrVyKyldhv6lMb49mJtanaraN\nYAkyr8FVrJAOcIcEkegCpOcqGmOtr0amL5WE+rPZQm6enN2thes95tr9OXEE\neE8nkHzQoYUB6gkY502u2db7NWMld2n0Um3S1ez9zKpK0K7EL7HARHc0eDB5\nEZjORHlrWdnyw5L2VeCjPivRYFRZVIZKqJctFJV2KJhQAkkehjPxJ9BqgrNS\n2y3KLa1Eq+4suWy/P0ICmnWnCoIUIJE5jyImRIxg60YpWR2ijKfGWmWE2LRX\n2hqruxQ0A2B44EEHgD1yTAA5PGdZ7UuWjQSjGG9kh6ddP+oV/D9z2zUu2GwM\n4qrtUkMoKfIAKxJBJAgrM4AOrkKIT7mULsuK/wAcWPTtdsbZtUbTtBxFyury\nz+JvNcCoqXOVpSZJJIOTJMzjnTLbVFcYBRjzs/PoxepaVS01v7QeuNe4ff8A\nFEmTghABhPpABMYJMDWbZdP0zZqqjWtobbihVIpx69KFvpWQO9TiwklRPEGT\nPz5xGeFprlY/p2R3310feTIRdd2sVBNNs60uPLMpNbVAhJB8wDk8giYjiIOt\nfH8c292mBl+Z5LVP/wDRiO1q6+1oqLg69VPhJlCT3JAPlEAJHzgZ+etF21Yi\n1EyVRdnPcuyXWfp82lTSriOxowVstHJzwVekCIA+R1mX+Sb/AANnH8Qod2Fs\nWC22egpQzZ6Nun7h3KQge+oxEqJkk85JJ+Osmdjte5mpGpVLVZAt62y5XDdd\nI+9TPtVKFhuh8NMYCiUlJOFHuIJMwPTGdLHeq9Es4RS6LUtlqr27WwbmtJqQ\nhIfXISO8CCMQJ8sY9BGs6S/yMqSslH0D9K20kqVJicqwBHnAE/kPnpeAz5mx\norHXClSTKwP5ThHOJA5+ZnSqIvLYyVhqagdqUleMpBwM+YmMfHPnp2h6kNtR\nZap2FPvhIOSE5I+pxpUx4iVbqejJUgDvyO5RlRnynScpP2OUdCR5twrHYntM\nzmZP0GdOSX7FOD9ekni4aADQAaAOzNk0hRYahXYffqlR8IQmP11wF8tyR6jx\n+pfHSXaVFfdsVP4xxxoh0BtQAKZkk9w8+BABGdMhkOmTKGTXzibLz0/25XXB\n+zVrQDyUBZWhBAKPeIJPBHuHE4g+mdTHzYy/8M2zEmu12VdvT2YGqxpdRaEB\nxZlSVNAAkmCAR5mDPn89aVeQ/wDr2U51Rl1Ipyv6b7/6fXJu52lytpqmjcC2\nqmmcU060ocKBBBB+IM6kscLupIZFWUf62XB0x9tjfu1SizdRrWNw0iFBP4oH\nwK5pOBPcB2OACTC0hSjysCdZeR4qFncOi/R5GUerTsDpX7R/T/qShprbm5Gn\nahYHdbq6GatGCSAgmFwBJKCoAcmdZFtF2P8Aki5xqyfw9ljV7W2d1Uxt9zpK\nd1JHcWKhKVgHIChiAeYIII9RqtJRmuwgsrGfvo5+65eylQ7ppFXLY9Whitpi\not0lUslCkST2JdAkEYACgRMkqA0+q74X36NKnOdn1tXZSdh2ruuxUq9tbts1\nTRXKgPalDyY8VqYCm1iUuJBlPckkYGcjUGRLm+UTZx5RS6Kr6uVVlcuSKalW\n1U1rQUh8N5DRHBKxI7gSQQCSCMxOtfAi4x+xn590OW4lh9KbV0/tm36Dcm5a\np/c91QyVt2lAUG6dQBEOcESQRjKgP4uCHXThTLZFUrciOkx+3Rv++7xdbar3\nWmaKmHYzQU6ShhhH+XsECcZkCYzJzrPtvnPs0cfCpre/2Qu7WG2rbVVUj6KJ\nYIUUrPuH4A4j/eMabVZK16SLFvChcpvoy2xW39lDrG37aKmpEgVbvDU8kkEC\nIPqY5Ppq6sHb3YZF/mFFfHV2YP7due4rmTdX6m/V7Ugkq/dNkySCTCUjBxiT\nwCdXHZTix1rRmRpyMqXb2Te39IxTtofvVQhxojDVGSG/WFLgKJ5kCIg886y7\n/KyfUDTxvEQi92EiasdFTNCkpKJtLaT/AANABIPqY8/mZP56o/NK3uxmsq41\nLVaNZ266FFQTCOYTMjzyeT5cR5aVSS9BKx6HChsz1GouU7cKOUiJBJHmOYOM\nnTlHn7K7t0xLV7Qq7nWN1FXT1FS6pRA7ySODwOEiTwPnrRpWo6J5SWiYWrbF\nfQ29LCgC4Z7EqklsGDAOfQkx6/eJ1/Yysi6UX0ZVO2KhUqUgIIGSqSZ+H+xp\nso6Io5CkMtbYKdoKS8kKkEHuIIOeI41BKWizF8uxnfoAyPCbQEhOACI+nrpn\nLZMhprKdKSfEkD7A6VdkiZH61+nSk+C0Vk/5QAPuefvp0VL9juYxVT9a7Hhk\nNJOPdEkj5+X2Opkor2I5HCWvRjxwNABoANAHcm0qYo22AeTVOHnkdqR/TXnl\nkvueqyf10dV9A9rvu9PkVyacrDtU7EZMCBxp8a+cdmNmWqMtFludPNv7vbp6\nesSwzV0yFtFLpKAUklUgqMAhUQCkyFKyT2jVpYzmvq9FWvOljv8AlEFtGybl\nty3sUFXb1Jph3uMpXEtkuK7h7p90EyoAgAgpMGcQP5sZ9MuT+DKXXsTXfp/a\nb4kuVLXe4UkEuEzxj4H4YGrVecpPVhSsxnD8CouoHs02+6kuUdCFkzAAAUCR\n5ETn4g6vxv1+LKrTX5ooPdXQjcG23y9QJdloylK5Ch6QRnU7sjYtWIbqSe62\nSjZftL9W+nzzdr3S47uCgbPaWq9RL6RMnsfgqmMDv7gBAAEaoX+Mrv7g9F+j\nPnR1Ytl8be9rfbl4ommrVVVZrl9qTb7gA2QsxPYsEhQkkASDAntEwMO/x+RT\n0+0a9GTjZL3Hpizd22t57+tztbuq8GzNvAfhrcySH6g89iUjJkAiODEQc6bX\nQqPtIvTui1wr9lZsexnenbLcbvV7gordXhJVRW5890JAn966mUpIAMAAgzJI\nggW/+TUmoVlVY6T3NEU2jYr7sayX9zcNtfpEsMC4sqrJQ2632KAKVn3VJkJE\npJAJA+Gq2T/9TNJezRqnXj1/bohF+6y2GtuVNb9t0aH6p1aG3K5xRQwySYVk

[["house_data_png/bath_1111.png", "house_data_png/bath_1149.png", "house_data_png/bath_1141.png", "house_data_png/bath_1136.png", "house_data_png/bath_1163.png"], [0.9867925643920898, 0.9853135347366333, 0.9835642576217651, 0.9810572862625122, 0.980038046836853]]